In [ ]:
import pandas as pd
import ewtpy
import pywt
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras import initializers

In [ ]:
df1 = pd.read_csv('data_day.csv')
df2 = pd.read_csv('data_2018-08-05_2021-04-30.csv')
df3 = pd.read_csv('data_2018-05-05_2018-08-04.csv')
df4 = pd.read_csv('data_2015-08-09_2018-05-04.csv')
df5 = pd.read_csv('data_2013-02-12_2015-08-08.csv')
df = pd.concat([df5, df4, df3, df2, df1])
print(df.to_string())
data = df.to_numpy()
temp = df['temp'].to_numpy()
temp_max = df['tempmax'].to_numpy()
temp_min = df['tempmin'].to_numpy()

dew = df['dew'].to_numpy()
humidity = df['humidity'].to_numpy()


precipitation = df['precip'].to_numpy()
precipcover = df['precipcover'].to_numpy()
precipprob = df['precipprob'].to_numpy()

windspeed = df['windspeed'].to_numpy()
winddir = df['winddir'].to_numpy()

cloudcover = df['cloudcover'].to_numpy()
solar_radiation = df['solarradiation'].to_numpy()
solarenergy = df['solarenergy'].to_numpy()
uvindex = df['uvindex'].to_numpy()
moonphase = df['moonphase'].to_numpy()

training_set = np.array([humidity, temp, precipitation, precipprob, windspeed, solar_radiation])
plt.plot(humidity)

NUM = 0

In [ ]:

training_set_tp = training_set.transpose()

In [ ]:
def window_data(data, input_size, output_size):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.window(input_size + output_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(input_size + output_size))
    X = dataset.map(lambda window: (window[:-output_size]))
    y = dataset.map(lambda window: (window[-output_size:]))
    return X,y

In [ ]:
X_tf,y_tf = window_data(training_set_tp, 200, 1)
X = np.array([x.numpy() for x in X_tf]).transpose(0,2,1)
y = np.array([y.numpy() for y in y_tf]).transpose(0,2,1)
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True
)


In [ ]:
model = Sequential(
    [
        tf.keras.Input(shape=(6,200,)),
        tf.keras.layers.Dense(units = 25, activation = "relu", kernel_regularizer=tf.keras.regularizers.L1(0.005)),
        tf.keras.layers.Dense(units = 15, activation = "relu", kernel_regularizer=tf.keras.regularizers.L1(0.005)),
        tf.keras.layers.Dense(units = 10, activation = "relu", kernel_regularizer=tf.keras.regularizers.L1(0.005)),
        tf.keras.layers.Dense(units = 10, activation = "linear")
        ### END CODE HERE ### 
    ], name = "mlpnn_model" 
) 
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
)
print(X_train.shape)
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
hist_mlpnn = model.fit(
    X_train,y_train,
    epochs=1000,  
    # callbacks = [callback] 
) 

In [ ]:
# model = Sequential(
#     [
#         tf.keras.Input(shape=(5,)),
#         tf.keras.layers.LSTM(units = 32),
#     ], name = "lstm_model" 
# ) 
# model.compile(
#     loss=tf.keras.losses.MeanSquaredError(),
#     optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
# )
# if (NUM != 0):
#     X_train = X_train[:-NUM]
# else:
#     X_train = X_train[:]
# print(X_train.shape)
# hist_mlpnn = model.fit(
#     X_train,y_train,
#     epochs=500,  
# ) 

In [ ]:
# model = Sequential(
#     [
#         tf.keras.Input(shape=(5,)),
#         tf.keras.layers.Dense(units = 2000, activation = "relu",kernel_initializer=initializers.RandomNormal(stddev=0.01),
#     bias_initializer=initializers.Zeros()),
#         tf.keras.layers.Dense(units = 1, activation = "linear")
#         ### END CODE HERE ### 
#     ], name = "elm_model" 
# ) 
# model.compile(
#     loss=tf.keras.losses.MeanSquaredError(),
#     optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
# )
# # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
# if (NUM != 0):
#     X_train = X_train[:-NUM]
# else:
#     X_train = X_train[:]
# hist_elm = model.fit(
#     X_train,y_train,
#     epochs=1000,   
#     # callbacks = [callback]
# ) 


In [ ]:
yt_predict = model.predict(X_train)
y_predict = model.predict(X_test)
print(yt_predict.shape)
print(y_train.shape)
feature = 0
for i in range(1):
    print(f"TRAIN {i+1}")
    print(f"R: {np.corrcoef(yt_predict[:,feature,i],y_train[:,feature,i])}")
    print(f"RMSE: {mean_squared_error(y_train[:,feature,i],yt_predict[:,feature,i])**(1/2)}")
    print(f"MAE: {mean_absolute_error(y_train[:,feature,i],yt_predict[:,feature,i])}")
for i in range(1):
    print(f"TRAIN {i+1}")
    print(f"R: {np.corrcoef(y_predict[:,feature,i],y_test[:,feature,i])}")
    print(f"RMSE: {mean_squared_error(y_test[:,feature,i],y_predict[:,feature,i])**(1/2)}")
    print(f"MAE: {mean_absolute_error(y_test[:,feature,i],y_predict[:,feature,i])}")
plt.scatter(y_test[:,feature,0],y_predict[:,feature,0])

In [ ]:
yt_predict = model.predict(X_train)
y_predict = model.predict(X_test)
plt.scatter(yt_predict[:,0],y_train[:,0])
plt.scatter(y_predict[:,0],y_test[:,0])
for i in range(len(y_predict) - NUM):
    print(f"predict: {y_predict[i,0]}, test: {y_test[i, 0]}")

print("TRAIN DETAIL")
print(f"R: {np.corrcoef(yt_predict[:,0],y_train[:,0])}")
print(f"RMSE: {mean_squared_error(y_train[:,0],yt_predict[:,0])**(1/2)}")
print(f"MAE: {mean_absolute_error(y_train[:,0],yt_predict[:,0])}")
# print(f"SCORE: {accuracy_score(y_train[:,0],yt_predict[:,0])}")
mt, bt = np.polyfit(yt_predict[:,0],y_train[:,0], 1)
plt.plot(yt_predict[:,0], mt*yt_predict[:,0]+bt, color = "yellow")
#add linear regression line to scatterplot 
print("TEST DETAIL")
print(f"R: {np.corrcoef(y_predict[:,0],y_test[:,0])}")
print(f"RMSE: {mean_squared_error(y_test[:,0],y_predict[:,0])**(1/2)}")
print(f"MAE: {mean_absolute_error(y_test[:,0],y_predict[:,0])}")
# print(f"SCORE: {accuracy_score(y_test[:,0],y_predict[:,0])}")
m, b = np.polyfit(y_predict[:,0],y_test[:,0], 1)
plt.plot(y_predict[:,0], m*y_predict[:,0]+b, color = "red")


In [ ]:
# plt.plot(history.history['loss'][200:500])
# print(history.history["loss"][400:500])
plt.plot(hist_mlpnn.history['loss'][500:1000])

In [ ]:
plt.plot(y_predict, label = "predict")
plt.plot(y_test)
plt.plot(abs(y_predict - y_test))
plt.legend(['predict', 'test', 'loss'])